In [1]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

### Load iris Dataset

In [2]:
iris_dict = load_iris()
X = iris_dict['data']
y = iris_dict['target']

In [3]:
# shuffle arrays since y values are in order

from sklearn.utils import shuffle
X_new, y_new = shuffle(X, y, random_state=0)

In [4]:
# Divide samples into train and test 

In [5]:
n_samples_train = 120 # number of samples for training (--> #samples for testing = len(y_new) - 120 = 30)
X_train = X_new[:n_samples_train, :]
y_train = y_new[:n_samples_train]

X_test = X_new[n_samples_train:, :]
y_test = y_new[n_samples_train:]

#### Fit logistic regression model

In [6]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [7]:
y_pred = clf.predict(X_test)

#### Metrics

In [8]:
from sklearn.metrics import accuracy_score

In [9]:
accuracy_score(y_test, y_pred)

0.9333333333333333

#### Save Model

In [10]:
import pickle

In [11]:
with open('container/iris_trained_model.pkl', 'wb') as f:
    pickle.dump(clf, f)

In [12]:
with open('container/iris_trained_model.pkl', 'rb') as f:
    clf_loaded = pickle.load(f)

In [13]:
clf_loaded

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
import boto3

session = boto3.Session(profile_name='pietto')
s3_client = session.client('s3')

with open('container/iris_trained_model.pkl', 'rb') as data:
    s3_client.upload_fileobj(data, 'mlsagemakertemp', 'ml/iris_trained_model.pkl')

In [ ]:
%%sh

algorithm_name=ml-model

cd container #your directory with dockerfile and other sm components

account=$(aws sts get-caller-identity --query Account --output text)

# Region, defaults to us-west-2
region=$(aws configure get region)
region=${region:-us-west-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"



# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
import boto3

client = boto3.client('ecs')

response = client.create_cluster(
    clusterName='Ml-test',
    tags=[
        {
            'key': 'ClusterName',
            'value': 'Ml-test'
        },
    ],
    settings=[
        {
            'name': 'containerInsights',
            'value': 'disabled'
        },
    ],
    configuration={
        'executeCommandConfiguration': {
            'logging': 'NONE'
        }
    },
    capacityProviders=[
        'FARGATE',
    ]
)

In [ ]:
import boto3

algorithm_name = "ml-model"

account_id = boto3.client('sts').get_caller_identity().get('Account')

client = boto3.client('ecs')

region_name = client.meta.region_name

image_name = f"{account_id}.dkr.ecr.{region_name}.amazonaws.com/{algorithm_name}"

response = client.register_task_definition(
    family='Ml-test-td',
    executionRoleArn=f"arn:aws:iam::{account_id}:role/ecsTaskExecutionRole",
    networkMode='awsvpc',
    containerDefinitions=[
        {
            "name": "fargate-app", 
            "image": image_name, 
            "portMappings": [
                {
                    "containerPort": 80, 
                    "hostPort": 80, 
                    "protocol": "tcp"
                }
            ], 
            "essential": True, 
            "entryPoint": [], 
            "command": ["null"],
        }
    ],
    volumes=[],
    placementConstraints=[],
    requiresCompatibilities=['FARGATE'],
    cpu='512',
    memory='1024',
    tags=[
        {
            'key': 'TaskDef',
            'value': 'Ml-test-td'
        },
    ]
)

taskdefarn = response['taskDefinition']['taskDefinitionArn']

print(taskdefarn)

In [ ]:
# Run tasks

import boto3

client = boto3.client('ecs')

response = client.run_task(
    cluster='Ml-test',
    count=1,
    taskDefinition=taskdefarn,
    launchType='FARGATE',
    networkConfiguration={
        'awsvpcConfiguration': {
            'subnets': [
                'subnet-0edafcde18b6c3ebd',
            ],
            'securityGroups': [
                'sg-0859bcea2aa43ecd5',
            ],
            'assignPublicIp': 'ENABLED'
        }
    },
    tags=[
        {
            'key': 'TaskDef',
            'value': 'Ml-test-td'
        },
    ]
)